# REINFORCE in PyTorch

При создании ноутбука использовались материалы курса [Practical RL](https://github.com/yandexdataschool/Practical_RL): [reference link](https://github.com/yandexdataschool/Practical_RL/blob/master/week06_policy_based/reinforce_pytorch.ipynb)

В данном задании мы создадим модель для игры `CartPole-v0` с иcпользованием policy gradient (REINFORCE).

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mryab/dl-hse-ami/blob/main/week12_nondiffnets/homework.ipynb)

In [ ]:
import sys, os
if 'google.colab' in sys.modules and not os.path.exists('.setup_complete'):
    !wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/setup_colab.sh -O- | bash
    !touch .setup_complete

# This code creates a virtual display to draw game images on.
# It will have no effect if your machine has a monitor.
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY")) == 0:
    !bash ../xvfb start
    os.environ['DISPLAY'] = ':1'

Starting virtual X frame buffer: Xvfb.


In [ ]:
# install old version 
!pip install -q gym[classic_control]==0.15.3

In [ ]:
import gym
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Потенциальная проблема: с некоторыми версиями `pyglet` следующая ячейка может упасть с ошибкой `NameError: name 'base' is not defined`. Ссылка на описание проблемы на GitHub находится [здесь](https://github.com/pyglet/pyglet/issues/134). Если Вы наблюдаете такую проблему, попробуйте перезапустить kernel.

In [ ]:
env = gym.make("CartPole-v0")

# gym compatibility: unwrap TimeLimit
if hasattr(env, '_max_episode_steps'):
    env = env.env

env.reset()
n_actions = env.action_space.n
state_dim = env.observation_space.shape

plt.imshow(env.render("rgb_array"))

# Создание сети для REINFORCE

Для алгоритма REINFORCE нам понадобится модель, которая предсказывает вероятность действий по состоянию среды.

Для численной стабильности, пожалуйста, __не используйте softmax-слой в архитектуре модели__.
Мы будем использовать softmax или log-softmax там, где это нужно.

**Задание 1 (0.3 балла).** Реализуйте простую сеть-агента, принимающую на вход состояние среды и выдающую логиты для каждого действия.

In [ ]:
import torch
import torch.nn as nn

In [ ]:
# Build a simple neural network that predicts policy logits. 
# Keep it simple: CartPole isn't worth deep architectures.
model = nn.Sequential(
  <YOUR CODE: define a neural network that predicts policy logits>
)

### Функция предсказания верояностей

**Задание 2 (0.1 балла).**  Реализуйте функцию, выдающую вероятности действий при условии состояний.

Заметка: выход этой функции не torch tensor, а numpy array. Поэтому нам не нужно считать в ней градиенты.
<br>
Используйте [no_grad](https://pytorch.org/docs/stable/autograd.html#torch.autograd.no_grad),
чтобы отключить подсчет градиентов.
<br>
Также можно использовать `.detach()` (или `.data`), но эти два способа имеют следующие отличия:
<br>

*  При использовании `.detach()` граф вычислений строится, но затем отсоединяется от конкретного тензора, поэтому `.detach()` нужно использовать, если граф вычислений нужен нам для подсчета градиента через какой-то другой (не тот, на котором мы вызвали `.detach()`) тензор

*   При использовании `no_grad()`, PyTorch не строит графа, поэтому в этой задаче данный способ предпочителен



In [ ]:
def predict_probs(states):
    """ 
    Predict action probabilities given states.
    :param states: numpy array of shape [batch, state_shape]
    :returns: numpy array of shape [batch, n_actions]
    """
    # convert states, compute logits, use softmax to get probability
    <YOUR CODE>
    return <YOUR CODE>

In [ ]:
test_states = np.array([env.reset() for _ in range(5)])
test_probas = predict_probs(test_states)
assert isinstance(test_probas, np.ndarray), \
    "you must return np array and not %s" % type(test_probas)
assert tuple(test_probas.shape) == (test_states.shape[0], env.action_space.n), \
    "wrong output shape: %s" % np.shape(test_probas)
assert np.allclose(np.sum(test_probas, axis=1), 1), "probabilities do not sum to 1"

### Сыграем в игру!

Теперь мы можем использовать построенную модель, чтобы сыграть в игру!

**Задание 3 (0.1 балла).** Заполните пропуски в коде одной сессии.

In [ ]:
def generate_session(env, t_max=1000):
    """ 
    Play a full session with REINFORCE agent.
    Returns sequences of states, actions, and rewards.
    """
    # arrays to record session
    states, actions, rewards = [], [], []
    s = env.reset()

    for t in range(t_max):
        # action probabilities array aka pi(a|s)
        action_probs = predict_probs(np.array([s]))[0]

        # Sample action with given probabilities.
        a = <YOUR CODE>
        new_s, r, done, info = env.step(a)

        # record session history to train later
        states.append(s)
        actions.append(a)
        rewards.append(r)

        s = new_s
        if done:
            break

    return states, actions, rewards

In [ ]:
# test it
states, actions, rewards = generate_session(env)

### Подсчет кумулятивной награды

$$
\begin{align*}
G_t &= r_t + \gamma r_{t + 1} + \gamma^2 r_{t + 2} + \ldots \\
&= \sum_{i = t}^T \gamma^{i - t} r_i \\
&= r_t + \gamma * G_{t + 1}
\end{align*}
$$

**Задание 4 (0.2 балла).** Реализуйте функцию, подсчитывающую кумулятивную награду с дисконтом $\gamma$.

In [ ]:
def get_cumulative_rewards(rewards,  # rewards at each step
                           gamma=0.99  # discount for reward
                           ):
    """
    Take a list of immediate rewards r(s,a) for the whole session 
    and compute cumulative returns (a.k.a. G(s,a) in Sutton '16).
    
    G_t = r_t + gamma*r_{t+1} + gamma^2*r_{t+2} + ...

    A simple way to compute cumulative rewards is to iterate from the last
    to the first timestep and compute G_t = r_t + gamma*G_{t+1} recurrently

    You must return an array/list of cumulative rewards with as many elements as in the initial rewards.
    """
    <YOUR CODE>
    return <YOUR CODE: array of cumulative rewards>

In [ ]:
get_cumulative_rewards(rewards)
assert len(get_cumulative_rewards(list(range(100)))) == 100
assert np.allclose(
    get_cumulative_rewards([0, 0, 1, 0, 0, 1, 0], gamma=0.9),
    [1.40049, 1.5561, 1.729, 0.81, 0.9, 1.0, 0.0])
assert np.allclose(
    get_cumulative_rewards([0, 0, 1, -2, 3, -4, 0], gamma=0.5),
    [0.0625, 0.125, 0.25, -1.5, 1.0, -4.0, 0.0])
assert np.allclose(
    get_cumulative_rewards([0, 0, 1, 2, 3, 4, 0], gamma=0),
    [0, 0, 1, 2, 3, 4, 0])
print("looks good!")

looks good!


### Функция потерь и обучение

Теперь нам нужно определить целевую функцию и обновление нашего policy gradient.

Наша целевая функция задается вот так:

$$ J \approx  { 1 \over N } \sum_{s_i,a_i} G(s_i,a_i) $$

REINFORCE дает нам способ подсчитать градиент математического ожидания награды по параметрам policy. Формула выглядит так:

$$ \nabla_\theta \hat J(\theta) \approx { 1 \over N } \sum_{s_i, a_i} \nabla_\theta \log \pi_\theta (a_i \mid s_i) \cdot G_t(s_i, a_i) $$

Мы можем использовать возможности PyTorch для автоматического дифференцирования, задав нашу целевую функцию таким образом:

$$ \hat J(\theta) \approx { 1 \over N } \sum_{s_i, a_i} \log \pi_\theta (a_i \mid s_i) \cdot G_t(s_i, a_i) $$

Когда мы подсчитаем градиент этой функции по отношению к весам $\theta$, он станет в точности равен нашему policy gradient.

**Задание 5 (0.3 балла).** Заполните пропуски в функции `train_on_session`, реализовав вычисление функции потерь и шаг градиентного спуска.

In [ ]:
def to_one_hot(y_tensor, ndims):
    """ helper: take an integer vector and convert it to 1-hot matrix. """
    y_tensor = y_tensor.type(torch.LongTensor).view(-1, 1)
    y_one_hot = torch.zeros(
        y_tensor.size()[0], ndims).scatter_(1, y_tensor, 1)
    return y_one_hot

In [ ]:
# Your code: define optimizers
optimizer = torch.optim.Adam(model.parameters(), 1e-3)


def train_on_session(states, actions, rewards, gamma=0.99, entropy_coef=1e-2):
    """
    Takes a sequence of states, actions and rewards produced by generate_session.
    Updates agent's weights by following the policy gradient above.
    Please use Adam optimizer with default parameters.
    """

    # cast everything into torch tensors
    states = torch.tensor(states, dtype=torch.float32)
    actions = torch.tensor(actions, dtype=torch.int32)
    cumulative_returns = np.array(get_cumulative_rewards(rewards, gamma))
    cumulative_returns = torch.tensor(cumulative_returns, dtype=torch.float32)

    # predict logits, probas and log-probas using an agent.
    logits = model(states)
    probs = nn.functional.softmax(logits, -1)
    log_probs = nn.functional.log_softmax(logits, -1)

    assert all(isinstance(v, torch.Tensor) for v in [logits, probs, log_probs]), \
        "please use compute using torch tensors and don't use predict_probs function"

    # select log-probabilities for chosen actions, log pi(a_i|s_i)
    log_probs_for_actions = torch.sum(
        log_probs * to_one_hot(actions, env.action_space.n), dim=1)
   
    # Compute loss here. Don't forgen entropy regularization with `entropy_coef` 
    entropy = <YOUR CODE>
    loss = <YOUR CODE>

    # Gradient descent step
    <YOUR CODE>

    # technical: return session rewards to print them later
    return np.sum(rewards)

### Обучение

In [ ]:
for i in range(100):
    rewards = [train_on_session(*generate_session(env)) for _ in range(100)]  # generate new sessions
    
    print("mean reward:%.3f" % (np.mean(rewards)))
    
    if np.mean(rewards) > 500:
        print("You Win!")  # but you can train even further
        break

### Результаты & видео

In [ ]:
# Record sessions

import gym.wrappers

with gym.wrappers.Monitor(gym.make("CartPole-v0"), directory="videos", force=True) as env_monitor:
    sessions = [generate_session(env_monitor) for _ in range(100)]

In [ ]:
# Show video. This may not work in some setups. If it doesn't
# work for you, you can download the videos and view them locally.
# You may face 'No video with supported format and MIME type found' issue

from pathlib import Path
from IPython.display import HTML

video_names = sorted([s for s in Path('videos').iterdir() if s.suffix == '.mp4'])

HTML("""
<video width="640" height="480" controls>
  <source src="{}" type="video/mp4">
</video>
""".format(video_names[-1]))  # You can also try other indices